# Parameters

In [1]:
model_file          = "../models/europe.json"
load_series_file    = "data/loads.csv"
noise_series_file   = "data/cost_noise.csv"
nuclear_series_file = "data/nuclear_2016.csv"
border_flows_file   = "data/border_flows_2016.csv"
output_dir          = "data/TOPF_run";

# Network

In [2]:
using TemperateOptimalPowerFlow
using CSV, DataFrames, DataDrop

Import the model:

In [3]:
network = import_model(model_file)

Dict{String, Dict{String, Any}} with 4 entries:
  "gen"    => Dict("1"=>Dict{String, Any}("entsoe_names"=>Any[], "model"=>2, "a…
  "branch" => Dict("4304"=>Dict{String, Any}("br_r"=>0.00174284, "rate_a"=>17, …
  "load"   => Dict("4304"=>Dict{String, Any}("load_bus"=>4304, "status"=>1, "qd…
  "bus"    => Dict("4304"=>Dict{String, Any}("coord"=>Any[11.8433, 47.3314], "n…

# Assign ramp constraints

Add ramp constraint by defining a maximum ramp rate of 200 MW / hour for coal generators 

In [4]:
for gen ∈ values(network["gen"])
    gen["max_ramp_rate"] = gen["aggregated_type"] == "coal" ? 2.0 : 0.0
end

# Nuclear production

Import actual time series for nuclear generators:

In [5]:
nuc_series_df = CSV.read(nuclear_series_file, DataFrame)
nuc_series_df.id .= string.(nuc_series_df.id);

Separate nuclear generators as non-dispatchable:

In [6]:
split_nondispatchable!(network, nuc_series_df.id)

Export the time series:

In [7]:
filter!(row -> string(row.id) in keys(network["gen_nd"]), nuc_series_df)

Row,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,⋯
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,79,1970.42,1970.52,1970.62,1970.62,1970.22,1968.94,1969.83,1969.93,1970.42,1969.63,1967.76,1965.79,1964.41,1963.04,1962.25,1962.45,1963.13,1960.67,1962.94,1963.04,1964.02,1964.51,1966.97,1966.97,1966.78,1964.41,1964.61,1964.71,1964.61,1963.43,1962.64,1962.94,1962.74,1963.04,1961.95,1960.58,1961.07,1961.07,1961.17,1961.17,1960.87,1961.36,1963.04,1965.5,1969.34,1969.83,1969.53,1970.42,1969.24,1967.76,1968.15,1967.47,1967.17,1967.07,1966.88,1967.07,1967.37,1967.17,1964.61,1961.95,1961.76,1959.79,1959.59,1958.41,1958.31,1959.79,1961.36,1961.56,1962.54,1963.13,1964.22,1960.08,1956.05,1957.23,1957.43,1956.44,1953.68,1953.29,1952.6,1956.15,1967.37,1968.25,1970.22,1964.02,1960.08,1958.61,1963.43,1956.05,1945.32,1943.35,1957.33,1956.15,1957.52,1957.92,1958.21,1956.44,1956.15,1954.08,1955.26,⋯
2,87,1062.4,1062.3,1062.9,1062.9,1063.6,1063.1,1063.1,1063.3,1062.9,1059.7,1061.1,1061.1,1061.1,1061.6,1062.1,1062.5,1062.6,1062.9,1063.3,1062.5,1062.8,1063.1,1062.8,1063.5,1064.1,1063.7,1063.8,1064.1,1065.0,1064.1,1064.8,1064.5,1065.1,1065.0,1064.9,1064.8,1065.1,1064.7,1065.4,1065.1,1064.8,1065.2,1064.9,1065.2,1064.8,1065.2,1065.2,1064.9,1064.6,1064.6,1065.5,1065.7,1065.9,1065.3,1065.2,1064.7,1065.6,1065.0,1065.6,1065.2,1065.2,1064.9,1065.1,1065.1,1064.6,1064.9,1064.5,1064.1,1063.6,1063.9,1063.9,1063.3,1064.5,1063.6,1063.6,1062.9,1064.1,1063.5,1063.4,1063.6,1063.9,1063.7,1063.4,1062.6,1063.6,1063.1,1063.7,1063.2,1063.0,1063.1,1063.1,1062.7,1062.8,1063.7,1062.9,1063.6,1062.9,1064.2,1064.6,⋯
3,160,975.114,975.404,975.791,975.888,975.694,975.501,975.888,975.211,975.501,975.791,973.759,975.501,976.081,976.275,975.694,976.468,977.339,976.855,977.146,975.888,975.694,975.694,975.501,976.081,975.888,976.178,975.985,976.178,975.888,975.985,975.791,975.404,975.211,977.146,978.21,975.404,975.114,975.501,975.694,975.694,975.888,975.888,975.985,976.372,975.985,976.178,976.662,976.275,975.888,976.662,977.049,976.275,976.081,976.855,976.468,976.662,976.275,976.275,976.565,976.952,977.339,976.081,975.888,975.985,976.081,975.791,976.372,975.404,976.081,976.275,976.275,976.372,976.081,977.049,976.468,976.468,977.533,976.468,976.952,978.113,978.79,978.307,978.5,978.5,978.887,979.371,978.5,978.984,978.79,978.887,978.79,978.693,978.79,978.5,978.597,978.693,978.5,978.887,978.887,⋯
4,176,929.4,927.9,929.1,929.3,928.8,929.5,930.1,929.9,929.4,929.6,929.1,929.5,930.3,930.4,930.3,930.8,930.2,930.1,930.9,930.7,930.3,930.5,930.2,930.1,929.7,929.8,928.9,928.8,929.2,927.4,927.5,927.8,926.5,927.6,927.8,927.0,926.9,926.7,926.9,926.7,927.2,927.2,926.9,927.3,926.0,926.3,926.3,925.7,925.2,925.1,924.9,924.0,923.8,923.6,922.4,924.5,925.1,924.3,925.1,925.3,924.7,924.1,924.2,923.8,922.9,923.3,923.1,922.1,923.4,924.1,923.6,922.6,921.7,920.0,919.9,920.3,919.3,919.3,919.3,918.6,919.2,919.4,919.4,918.2,918.8,919.2,918.6,919.2,920.2,919.6,919.9,920.2,919.6,921.9,922.3,920.5,921.0,920.8,918.9,⋯
5,179,1914.31,1914

In [8]:
nuc_ids = nuc_series_df.id

44-element Vector{String}:
 "79"
 "87"
 "160"
 "176"
 "179"
 "209"
 "210"
 "232"
 "249"
 "256"
 "257"
 "265"
 "318"
 ⋮
 "723"
 "732"
 "751"
 "757"
 "782"
 "846"
 "908"
 "917"
 "921"
 "943"
 "950"
 "982"

In [9]:
nuc_series = Matrix(nuc_series_df[:, 2:end]) / 100.

44×8736 Matrix{Float64}:
 19.7042   19.7052   19.7062   19.7062   …  19.7042   19.7052   19.7052
 10.624    10.623    10.629    10.629       10.633    10.63     10.624
  9.75114   9.75404   9.75791   9.75888     10.4758    9.80338   9.75114
  9.294     9.279     9.291     9.293        9.299     9.296     9.296
 19.1431   19.1431   19.1431   19.1431      19.1431   19.1431   19.1431
 12.7429   12.736    12.477    12.7685   …  12.7764   12.7321   12.7311
 12.0824   12.0824   12.0787   12.0787      12.1026   12.1024   12.0844
 13.64     13.6173   13.553    13.53        13.6965   13.6178   13.635
 12.7271   12.7467   12.7437   12.7301      12.9856   12.9398   12.9096
 12.0348   12.0338   12.0309   12.023       12.0309   12.0328   12.0348
 12.8447   12.8466   12.8437   12.8437   …  12.8234   12.8251   12.8417
 14.0004   13.997    13.8992   13.7054      14.0317   14.0294   14.0088
 17.21     17.21     17.21     17.21        17.26     17.26     17.21
  ⋮                                      ⋱ 

# Loads

List of all countries in the network:

In [10]:
countries = Set([bus["country"] for bus ∈ values(network["bus"])])

Set{String} with 26 elements:
  "SI"
  "RO"
  "IT"
  "NL"
  "FR"
  "DK"
  "SK"
  "RS"
  "BG"
  "AT"
  "CH"
  "XX"
  "CZ"
  "ME"
  "DE"
  "MK"
  "BA"
  "BE"
  "AL"
  "LU"
  "HR"
  "ES"
  "GR"
  "HU"
  "PL"
  "PT"

### Compute expected production by country

Determine the total expected production:

In [11]:
expected_prod = Dict(country => 0.0 for country ∈ countries)

for gen ∈ values(network["gen"])
    expected_prod[gen["country"]] += gen["pexp"]
end

sort(expected_prod, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "DE" => 277.804
  "PL" => 135.462
  "FR" => 128.144
  "IT" => 126.413
  "ES" => 77.2725
  "NL" => 59.6867
  "AT" => 57.6037
  "RO" => 48.9335
  "GR" => 48.2076
  "PT" => 42.3998
  "CZ" => 35.3887
  "RS" => 29.6354
  "BG" => 19.8427
  "BE" => 17.3597
  "BA" => 13.4466
  "CH" => 11.6612
  "HU" => 10.6986
  "SI" => 10.1473
  "SK" => 9.12782
  "DK" => 7.71758
  "HR" => 6.54311
  "MK" => 5.00832
  "AL" => 4.43366
  "LU" => 3.56473
  "XX" => 2.6395
  "ME" => 2.33425

### Compute import/export balance by country

Export balance by country:

In [12]:
border_flows_df = CSV.read(border_flows_file, DataFrame)

Row,country,out_flow
,String3,Float64
1,HR,-717.674
2,RS,167.495
3,CH,-513.167
4,MK,-136.593
5,BE,-725.616
6,FR,3015.93
7,BG,189.706
8,RO,664.271
9,AT,-993.982


In [13]:
border_flows = Dict{String, Float64}()
for (country, flow) in eachrow(border_flows_df)
    border_flows[country] = flow / 100.0
end
border_flows["XX"] = 0.0

sort(border_flows, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "DE" => 56.6309
  "FR" => 30.1593
  "CZ" => 12.5109
  "RO" => 6.64271
  "PT" => 5.73057
  "BA" => 4.25852
  "PL" => 2.34598
  "BG" => 1.89706
  "RS" => 1.67495
  "SI" => 1.27665
  "AL" => 0.0469134
  "LU" => 0.0
  "XX" => 0.0
  "ME" => -0.309394
  "MK" => -1.36593
  "DK" => -2.81724
  "CH" => -5.13167
  "SK" => -5.37435
  "HR" => -7.17674
  "BE" => -7.25616
  "GR" => -8.38059
  "HU" => -9.63747
  "NL" => -9.91312
  "AT" => -9.93982
  "ES" => -14.5411
  "IT" => -41.3309

Check: total out flow adds up to zero

In [14]:
sum(values(border_flows))

4.440892098500626e-16

### Compute nuclear production by country

Nuclear production by country:

In [15]:
avg_nuclear = sum(nuc_series, dims=2) / size(nuc_series, 2)

nuclear_prod = Dict(country => 0.0 for country ∈ countries)
for i = 1:length(nuc_ids)
    country = network["gen_nd"][nuc_ids[i]]["country"]
    nuclear_prod[country] += avg_nuclear[i]
end
nuclear_prod

Dict{String, Float64} with 26 entries:
  "SI" => 0.0
  "ES" => 45.3376
  "GR" => 0.0
  "LU" => 0.0
  "HU" => 16.8633
  "CH" => 22.9463
  "MK" => 0.0
  "RS" => 0.0
  "XX" => 0.0
  "DE" => 90.667
  "AL" => 0.0
  "FR" => 441.027
  "PL" => 0.0
  "IT" => 0.0
  "AT" => 0.0
  "HR" => 0.0
  "PT" => 0.0
  "ME" => 0.0
  "NL" => 4.22129
  "BE" => 46.9143
  "RO" => 11.829
  "BG" => 17.1781
  "CZ" => 24.8885
  "SK" => 16.7945
  "DK" => 0.0
  "BA" => 0.0

### Compute total load by country

Determine annual load by country:

In [16]:
total_load = Dict(country => prod + nuclear_prod[country] - border_flows[country]
    for (country, prod) ∈ expected_prod)

sort(total_load, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "FR" => 539.011
  "DE" => 311.84
  "IT" => 167.744
  "ES" => 137.151
  "PL" => 133.116
  "NL" => 73.8211
  "BE" => 71.5302
  "AT" => 67.5436
  "GR" => 56.5882
  "RO" => 54.1199
  "CZ" => 47.7663
  "CH" => 39.7391
  "HU" => 37.1993
  "PT" => 36.6692
  "BG" => 35.1237
  "SK" => 31.2967
  "RS" => 27.9604
  "HR" => 13.7198
  "DK" => 10.5348
  "BA" => 9.18812
  "SI" => 8.87068
  "MK" => 6.37425
  "AL" => 4.38674
  "LU" => 3.56473
  "ME" => 2.64364
  "XX" => 2.6395

### Import loads

In [17]:
normalized_loads_df = CSV.read(load_series_file, DataFrame)
normalized_loads_df.id .= string.(normalized_loads_df.id)
sort!(normalized_loads_df, :id)

Row,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,⋯
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,100,0.642592,0.612423,0.571524,0.563611,0.579427,0.577297,0.582691,0.629311,0.699501,0.750834,0.74148,0.700062,0.606746,0.591304,0.588022,0.626869,0.707244,0.73449,0.725143,0.718051,0.658315,0.644565,0.604077,0.563212,0.546172,0.524276,0.514998,0.52677,0.57533,0.597634,0.623742,0.666755,0.674159,0.682643,0.679839,0.67546,0.666546,0.669611,0.655797,0.725183,0.804183,0.821639,0.813335,0.816906,0.741328,0.735141,0.702977,0.627301,0.609876,0.58208,0.551044,0.548369,0.55735,0.546802,0.603906,0.680013,0.74042,0.773317,0.796837,0.765817,0.679046,0.652577,0.651989,0.74762,0.860181,0.872862,0.866761,0.855133,0.798643,0.763649,0.704586,0.639765,0.613352,0.589711,0.575408,0.58918,0.623831,0.626123,0.734602,0.828889,0.92227,0.979104,1.0331,1.03355,0.960691,0.937385,0.892487,0.91211,0.959276,0.952526,0.92445,0.894341,0.835497,0.802214,0.746491,0.696189,0.678719,0.650095,0.635389,⋯
2,101,0.804738,0.765712,0.731958,0.72916,0.754654,0.793366,0.844983,0.928981,1.0064,1.06337,1.08695,1.06252,0.987648,0.98358,0.990667,1.03727,1.13167,1.14747,1.14514,1.10413,1.03093,0.972548,0.904002,0.812968,0.796145,0.757211,0.73276,0.74331,0.795152,0.865743,0.94644,1.03214,1.0794,1.12239,1.15192,1.14441,1.08729,1.09208,1.09048,1.1383,1.22729,1.23785,1.22563,1.17987,1.09427,1.03793,0.942622,0.850434,0.840637,0.809723,0.78394,0.794915,0.872331,0.995116,1.10745,1.21149,1.23866,1.23599,1.23007,1.22028,1.17571,1.18148,1.17616,1.24532,1.36739,1.37821,1.36034,1.28338,1.17667,1.09422,0.992654,0.891896,0.876482,0.855043,0.840586,0.853806,0.933702,1.05635,1.15729,1.26082,1.30858,1.32168,1.32649,1.32387,1.26546,1.27742,1.27321,1.3253,1.42182,1.42993,1.40612,1.32095,1.20687,1.12906,1.01893,0.932908,0.91834,0.883872,0.864605,⋯
3,1011,0.866559,0.822015,0.782458,0.754237,0.745761,0.75169,0.790912,0.831853,0.907548,0.985739,1.0422,1.09841,1.11475,1.09285,1.07995,1.06083,1.05415,1.13409,1.22099,1.24683,1.23765,1.19806,1.12728,1.01813,0.923583,0.856602,0.813733,0.794594,0.792359,0.807765,0.869774,0.965177,1.09194,1.20414,1.25546,1.28478,1.26344,1.24369,1.25653,1.22469,1.21626,1.26883,1.34131,1.36158,1.33277,1.28378,1.20181,1.07019,0.967182,0.900222,0.861961,0.840665,0.834638,0.842794,0.902311,1.01698,1.14664,1.23174,1.25265,1.27578,1.25336,1.23866,1.25924,1.23115,1.21501,1.25772,1.33509,1.35183,1.32387,1.27056,1.18983,1.07408,0.977737,0.907187,0.865144,0.845303,0.835786,0.852675,0.908318,1.02359,1.1545,1.25617,1.28131,1.30637,1.28129,1.26063,1.29193,1.24973,1.2317,1.27721,1.35569,1.37611,1.34714,1.28418,1.21931,1.09515,0.990178,0.920138,0.873599,⋯
4,103,0.728161,0.699862,0.682127,0.686873,0.751245,0.836888,0.926799,1.01,1.04444,1.03563,0.999959,0.992845,0.959748,0.985791,0.999294,1.0533,1.12384,1.13027,1.10647,1.04656,0.952567,0.888409,0.835993,0.755043,0.740572,0.718302,0.700297,0.701969,0.736053,0.77626,0.824239,0.884693,0.891503,0.907171,0.919622,0.913898,0.87652,0.889696,0.892554,0.958658,1.0603

Check that the IDs match:

In [18]:
bus_ids = normalized_loads_df.id
bus_ids == collect(keys(network["bus"]))

false

In [19]:
normalized_loads = Matrix(normalized_loads_df[:, 2:end])

4097×8736 Matrix{Float64}:
 0.642592  0.612423  0.571524  0.563611  …  0.771628  0.696678  0.627139
 0.804738  0.765712  0.731958  0.72916      0.988634  0.894836  0.810868
 0.866559  0.822015  0.782458  0.754237     1.14218   1.08928   0.996233
 0.728161  0.699862  0.682127  0.686873     0.878404  0.808906  0.730695
 0.775477  0.759438  0.758789  0.789805     0.947598  0.867312  0.784624
 0.726305  0.69671   0.682979  0.687546  …  0.859504  0.793883  0.734684
 0.833536  0.786514  0.749599  0.744651     1.00753   0.923315  0.833843
 0.90057   0.86819   0.841715  0.857524     1.13404   1.01131   0.916655
 0.807044  0.739449  0.706546  0.691963     1.03491   0.955588  0.876678
 0.689588  0.676876  0.669969  0.694314     0.84828   0.762904  0.702963
 0.439491  0.428502  0.425665  0.432145  …  0.493493  0.452914  0.423865
 0.898242  0.86778   0.838433  0.848181     1.11212   1.0132    0.902591
 0.893142  0.86283   0.839403  0.848966     1.11915   0.999422  0.9007
 ⋮                        

Check normalization:

In [20]:
sum(normalized_loads) / length(normalized_loads)

1.0001413983031382

Normalization by country:

In [21]:
normalized_annual_loads = Dict(country => [] for country ∈ countries)
for (id, annual_load) ∈ Dict(bus_ids .=> sum(normalized_loads, dims=2) / size(normalized_loads, 2))
    push!(normalized_annual_loads[network["bus"][id]["country"]], annual_load)
end

normalization = Dict(country => sum(annual_loads) / length(annual_loads)
    for (country, annual_loads) ∈ normalized_annual_loads)

Dict{String, Float64} with 26 entries:
  "SI" => 1.0
  "ES" => 1.0
  "GR" => 1.0
  "LU" => 1.00293
  "HU" => 0.999826
  "CH" => 1.0
  "MK" => 1.0
  "RS" => 1.0
  "XX" => 1.0
  "DE" => 1.00069
  "AL" => 1.0
  "FR" => 1.0
  "PL" => 1.0
  "IT" => 1.0
  "AT" => 1.00174
  "HR" => 1.0
  "PT" => 1.0
  "ME" => 1.0
  "NL" => 1.00025
  "BE" => 1.00043
  "RO" => 1.0
  "BG" => 1.0
  "CZ" => 1.0
  "SK" => 1.0
  "DK" => 1.0
  "BA" => 1.0

Check that the load assignement within each country is properly normalized:

In [22]:
load_prop_by_country = Dict(country => 0.0 for country in countries)
for bus ∈ values(network["bus"])
    load_prop_by_country[bus["country"]] += bus["load_prop"]
end

load_prop_by_country

Dict{String, Float64} with 26 entries:
  "SI" => 1.0
  "ES" => 1.0
  "GR" => 1.0
  "LU" => 1.0
  "HU" => 1.0
  "CH" => 1.0
  "MK" => 1.0
  "RS" => 1.0
  "XX" => 1.0
  "DE" => 1.0
  "AL" => 1.0
  "FR" => 1.0
  "PL" => 1.0
  "IT" => 1.0
  "AT" => 1.0
  "HR" => 1.0
  "PT" => 1.0
  "ME" => 1.0
  "NL" => 1.0
  "BE" => 1.0
  "RO" => 1.0
  "BG" => 1.0
  "CZ" => 1.0
  "SK" => 1.0
  "DK" => 1.0
  "BA" => 1.0

Compute a scaling value for each country:

In [23]:
load_scaling = [(
        network["bus"][id]["load_prop"]
        * normalization[network["bus"][id]["country"]]
        * total_load[network["bus"][id]["country"]]
        )
    for id ∈ bus_ids]

4097-element Vector{Float64}:
 0.8934261114738875
 1.5825000373038391
 0.813121818272138
 1.4753630936780373
 2.2254222971277824
 0.4372110373048939
 0.8807690040781996
 0.9598640799673998
 0.07712152835090184
 1.8597067898850632
 3.4263704978147866
 3.47088866099608
 3.5006348416166198
 ⋮
 0.1283089665347668
 0.19874288379779137
 0.3974857674630873
 0.21553571730621504
 0.19145321975182375
 0.4719580413098242
 0.9439160823050099
 0.12211186908659864
 0.19913420268885207
 0.398268405244948
 0.2072804507643949
 1.4718012201837631

In [24]:
loads = load_scaling .* normalized_loads

4097×8736 Matrix{Float64}:
 0.574108   0.547155   0.510614   …  0.689393   0.62243    0.560303
 1.2735     1.21174    1.15832       1.56451    1.41608    1.2832
 0.704618   0.668398   0.636234      0.928731   0.885717   0.810058
 1.0743     1.03255    1.00639       1.29596    1.19343    1.07804
 1.72576    1.69007    1.68863       2.10881    1.93014    1.74612
 0.317548   0.304609   0.298606   …  0.375785   0.347095   0.321212
 0.734153   0.692737   0.660224      0.887399   0.813227   0.734423
 0.864425   0.833345   0.807932      1.08853    0.970717   0.879864
 0.0622405  0.0570275  0.0544899     0.0798135  0.0736964  0.0676107
 1.28243    1.25879    1.24595       1.57755    1.41878    1.30731
 1.50586    1.46821    1.45849    …  1.69089    1.55185    1.45232
 3.1177     3.01197    2.91011       3.86003    3.51671    3.13279
 3.12657    3.02045    2.93844       3.91775    3.49861    3.15302
 ⋮                                ⋱                        ⋮
 0.108325   0.0980219  0.0934214  …

Check that the annual load matches the expectations:

In [25]:
sum(loads) / size(loads, 2) / sum(values(total_load))

1.0004002959745075

# Generation costs

In [26]:
gen_costs = Matrix(CSV.read(noise_series_file, DataFrame))

771×8736 Matrix{Float64}:
 -0.0262487    0.0198198    0.0790532   …  -0.0666133    0.0327033
 -0.0707873   -0.126037    -0.21975        -0.225453    -0.282577
  0.105176     0.069375     0.194085        0.072822     0.022119
 -0.0222022   -0.0534002   -0.150654       -0.0145013   -0.087602
 -0.0312321   -0.071578    -0.0364149      -0.0838126   -0.0406127
 -0.0341725    0.0172165    0.110258    …   0.00594135  -0.0312075
 -0.165547    -0.0400537    0.0516328       0.0739772   -0.0328689
 -0.134217    -0.191215    -0.132414       -0.132683    -0.0660482
 -0.196525    -0.0747628    0.0353276      -0.125528    -0.164061
  0.132592     0.00516041   0.0292117       0.15272      0.025195
 -1.53015e-5   0.120748     0.00266818  …  -0.032617    -0.027211
 -0.125775    -0.105427    -0.0152052       0.0769087    0.0717783
  0.171401     0.163038     0.144398        0.297562     0.0635374
  ⋮                                     ⋱                ⋮
 -0.00495856   0.0705164    0.0291467       0.0573

# Setup

In [27]:
setup(output_dir, network, loads, gen_costs, nuc_ids, nuc_series)

[2024-07-22 11:55:01]  Creating directory 'data/TOPF_run'
[2024-07-22 11:55:01]  Loading network
[2024-07-22 11:55:01]  Computing susceptance matrix
[2024-07-22 11:55:02]  Computing PTDF matrix (this may take some time)
[2024-07-22 11:55:55]  Distributing generator on buses
[2024-07-22 11:55:56]  Distributing loads on buses
[2024-07-22 11:55:56]  Distributing non-dispatchable generators on buses
[2024-07-22 11:55:56]  Computing max capacity for each generator
[2024-07-22 11:55:56]  Computing expected production for each generator
[2024-07-22 11:55:56]  Computing thermal limits
[2024-07-22 11:55:56]  Computing quadratic line costs
[2024-07-22 11:55:56]  Computing linear line costs
[2024-07-22 11:56:05]  Computing linear generation costs
[2024-07-22 11:56:05]  Computing total load constraints
[2024-07-22 11:56:05]  Computing ramp constraints
[2024-07-22 11:56:05]  Record generators IDs


In [28]:
readdir(output_dir)

17-element Vector{String}:
 "A_gen.h5"
 "A_load.h5"
 "A_nondispatch.h5"
 "A_ramp.h5"
 "PTDF_matrix.h5"
 "P_exp.h5"
 "P_load.h5"
 "P_max.h5"
 "P_nondispatch.h5"
 "P_total.h5"
 "gen_ids.h5"
 "linear_gen_cost.h5"
 "linear_line_cost.h5"
 "quadratic_cost.h5"
 "ramp_max.h5"
 "susceptance.h5"
 "thermal_limits.h5"

# List bus names associated with generators

This can be used to analyse the output:

In [29]:
gen_names = [network["bus"][string(network["gen"][id]["gen_bus"])]["name"] for id in get_ordered_gen_ids(network)];